In [69]:
!pip install -r requirements.txt

In [2]:
# Load data
import pandas as pd 

file_path='assets/datasets/clean/250604-consolidated_survey.xlsx'
df_load=pd.read_excel(file_path).drop(columns=['Unnamed: 0'])

In [2]:
df_load.head()

,start_date,end_date,survey_status,completion_level,completion_time,survey_completed,recorded_date,response_id,distribution_channel,user_language,...,soc_aspects,soc_aspects_other,es_assess,es_assess_other,es_more_info,es_certif,email,survey,inv_platform,inv_platform_other
0,2024-11-13 10:30:58,2024-11-13 10:37:24,0,100,386,1,2024-11-13 10:37:25.745,R_2CYvoda8TZhYFaN,anonymous,IT,...,"2,3,4",NaN,2,NaN,3.0,"2,3",NaN,BA,NaN,NaN
1,2024-11-14 14:00:16,2024-11-14 14:08:08,0,100,472,1,2024-11-14 14:08:09.160,R_2PCMoZEjgiOeupr,anonymous,IT,...,1,NaN,2,NaN,3.0,1,NaN,BA,NaN,NaN
2,2024-11-14 14:28:17,2024-11-14 14:36:18,0,100,480,1,2024-11-14 14:36:18.958,R_8HBDPZSIjz1M0rn,anonymous,IT,...,"1,3",NaN,3,NaN,5.0,"2,3",anmata@libero.it,BA,NaN,NaN
3,2024-11-14 16:46:00,2024-11-14 18:31:25,0,100,6324,1,2024-11-14 18:31:26.274,R_8dSEqbK9X2kHC6i,anonymous,IT,...,"1,4",NaN,2,NaN,3.0,1,NaN,BA,NaN,NaN
4,2024-11-14 18:26:48,2024-11-14 18:42:58,0,100,969,1,2024-11-14 18:42:58.484,R_2kq5hZdQfhR6sbI,anonymous,IT,...,"2,3",NaN,2,NaN,4.0,1,francoagostinoperone@gmail.com,BA,NaN,NaN


In [3]:
df_load.columns

Index(['start_date', 'end_date', 'survey_status', 'completion_level',
       'completion_time', 'survey_completed', 'recorded_date', 'response_id',
       'distribution_channel', 'user_language', 'recaptcha_score',
       'gender_male', 'age', 'children_num', 'edu_level', 'edu_field',
       'edu_field_other', 'working_status', 'working_status_other',
       'expertise_area', 'expertise_area_other', 'work_industries',
       'work_industries_other', 'work_company_size', 'roles_cat', 'inv_time',
       'inv_motivation', 'inv_motivation_other', 'ba_group', 'ba_group_name',
       'ba_group_name_other', 'ba_group_reason', 'ba_group_reason_other',
       'inv_num', 'inv_companies_last2y', 'inv_amount_average',
       'inv_capital_stake', 'inv_dd_duration', 'inv_criteria',
       'inv_criteria_other', 'inv_sectors', 'inv_sectors_other', 'inv_process',
       'inv_process_other', 'inv_visits', 'non_fin_contr',
       'non_fin_contr_other', 'child_bias', 'inv_goal', 'inv_exit',
       'inv_re

In [ ]:
# Proportions of women and men in both surveys

pd.options.display.max_columns = None
columns={
    'gender_male': 'gender',
    'start_date': 'percentage',
}
# df_temp=df_load[['survey', 'gender_male', 'age', 'children_num', 'edu_level']].groupby(['survey']).mean()
df_temp=df_load.copy()
df_temp['gender_male']=df_temp['gender_male'].replace(1, 'women')
df_temp['gender_male']=df_temp['gender_male'].replace(2, 'men')
df_temp=df_temp.dropna(subset=['gender_male'])
df_stats=(df_temp.groupby(['survey', 'gender_male'])['start_date'].count()).div(df_temp.groupby(['survey'])['start_date'].count())
df_stats=df_stats.reset_index().rename(columns=columns)
df_stats

,survey,gender,percentage
0,BA,men,0.712871
1,BA,women,0.287129
2,CI,men,0.617021
3,CI,women,0.382979


In [192]:
############### Functions ###############
import pandas as pd 
import scipy

# Returns the statistics in funcs on groups and variables columns
def basic_stats(df, funcs, groups, columns):
    df_results=pd.DataFrame()
    for column in columns:
        df_results=pd.concat([df_results,df.groupby(groups)[column].agg(funcs).reset_index().assign(var=column)])
    return df_results

# Mode or most frequent result 
def mode(x):
    return scipy.stats.mode(x.dropna())[0]

# Total number of records
def total(x):
    return len(x)

# Count the number of NA
def count_na(x):
    return x.isna().sum()

# Percentage of NA
def percent_na(x):
    return x.isna().sum()/len(x)

# Frequency table

def freq_table(df, column, start_index, category_labels):
    df_results=pd.DataFrame(columns=['Response', 'Frequency', 'Percentage'])
    for index, category_label in enumerate(category_labels):
        df_temp= pd.DataFrame([[category_label, len(df[df[column]==(index+start_index)]), round(100*len(df[df[column]==(index+start_index)])/len(df[column]),2)]], columns=df_results.columns)
        df_results=pd.concat([df_results,df_temp])
    df_results=pd.concat([df_results, pd.DataFrame([['NA', df[column].isna().sum(), round(100*df[column].isna().sum()/len(df[column]),2)]], columns=df_results.columns)])
    df_results=pd.concat([df_results, pd.DataFrame([['Total', len(df[column]), 100.00]], columns=df_results.columns)])
    return df_results

In [113]:
# Investor experience Numerical and ordinal variables
# Gender

funcs=[mode, 'median', 'mean', 'std', total, 'count', count_na, percent_na]
groups=['gender_male']
columns=['inv_time', 'inv_num', 'inv_amount_average', 'women_led_inv']

basic_stats(df_load, funcs, groups, columns)

,gender_male,mode,median,mean,std,total,count,count_na,percent_na,var
0,1.0,3.0,3.0,3.064516,1.093480,47,31,16,0.340426,inv_time
1,2.0,5.0,3.0,3.372093,1.337658,101,86,15,0.148515,inv_time
0,1.0,3.0,3.0,3.375000,1.539795,47,32,15,0.319149,inv_num
1,2.0,3.0,3.0,3.423529,1.483051,101,85,16,0.158416,inv_num
0,1.0,1.0,1.0,1.814815,1.177907,47,27,20,0.425532,inv_amount_average
1,2.0,1.0,2.0,2.500000,1.295408,101,74,27,0.267327,inv_amount_average
0,1.0,2.0,2.0,2.230769,1.394495,47,26,21,0.446809,women_led_inv
1,2.0,1.0,2.0,1.835616,1.027588,101,73,28,0.277228,women_led_inv


In [12]:
# Investor experience Numerical and ordinal variables
# survey

funcs=['mean', 'std', 'count']
groups=['survey']
columns=['inv_time', 'inv_num', 'inv_amount_average', 'women_led_inv']

basic_stats(funcs, groups, columns)

,survey,mean,std,count,var
0,BA,3.563830,1.187375,94,inv_time
1,CI,2.173913,1.029217,23,inv_time
0,BA,3.760870,1.329046,92,inv_num
1,CI,2.153846,1.347362,26,inv_num
0,BA,2.306818,1.271780,88,inv_amount_average
1,CI,2.384615,1.502135,13,inv_amount_average
0,BA,1.965517,1.175805,87,women_led_inv
1,CI,1.750000,0.866025,12,women_led_inv


In [277]:
from scipy.stats import mannwhitneyu
import pandas as pd 
import numpy as np
from scipy.stats import norm

def mw_test(df,columns, condition_label, condition_values):
    result_columns=['var', 'u_stat', 'pvalue']
    df_results=pd.DataFrame(columns=result_columns)
    for column in columns:
        
        df_temp=df.dropna(subset=[column, condition_label]) # Keep only non nan

        groups=[]
        for i in range(len(condition_values)):
            groups.append(df_temp[df_temp[condition_label]==condition_values[i]].loc[:,column].array)

        u, p=mannwhitneyu(groups[0], groups[1], alternative='two-sided')
        df_results=pd.concat([df_results, pd.DataFrame([[column, u, p]], columns=result_columns)])
    
    return df_results


In [299]:
# Mann Whitney test on gender_male

# columns=['inv_time', 'inv_num', 'inv_amount_average', 'women_led_inv']
columns=['inv_time']
condition_label='gender_male'
condition_values=[1,2]

display(mw_test(df_load, columns, condition_label, condition_values))

C:\Users\Dell\AppData\Local\Temp\ipykernel_13492\802478183.py:18: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



,var,u_stat,pvalue
0,inv_time,1155.5,0.261672


In [287]:
# Mann Whitney test on survey

# columns=['inv_time', 'inv_num', 'inv_amount_average', 'women_led_inv']
columns=['inv_time',]
condition_label='survey'
condition_values=['BA', 'CI']

mw_test(df_load, columns, condition_label, condition_values)

C:\Users\Dell\AppData\Local\Temp\ipykernel_13492\802478183.py:18: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



,var,u_stat,pvalue
0,inv_time,1729.5,0.000005


In [ ]:
import plotly.graph_objects as go

funcs=['mean', 'std', 'count']
groups=['gender_male']
columns=['inv_time', 'inv_num', 'inv_amount_average', 'women_led_inv']

df_stats=basic_stats(funcs, groups, columns)

categories=['Investment Experience', 'Number of Investments', 'Average Invested Amount', 'Number of Women-Led Investments']
r1=df_stats[df_stats[groups[0]]==1]['mean']
r2=df_stats[df_stats[groups[0]]==2]['mean']

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=r2,
      theta=categories,
      fill='toself',
      # fillcolor='rgba(0,0,255,0.2)',  # Example: 20% opacity
      # line_color='rgba(0,0,255,0.8)',
      opacity=0.5,
      line_shape='spline',
      name='Men'
))

fig.add_trace(go.Scatterpolar(
      r=r1,
      theta=categories,
      fill='toself',
      # fillcolor='rgba(255, 192,203,0.2)',  # Example: 20% opacity
      # line_color='rgba(255, 192,203,0.8)',
      opacity=0.5,
      line_shape='spline',
      name='Women',
))

axis_color = "rgba(108, 122, 137,1)"
fig.update_layout(
  polar=dict(
    bgcolor="rgba(0, 0, 0, 0)",
    angularaxis = dict(
    linewidth = 2,
    showline=True,
    linecolor =axis_color,
    gridcolor =axis_color,
  ),
    
  radialaxis = dict(
    side = "counterclockwise",
    showline = False,
    linewidth = 1,
    gridcolor =axis_color,
    gridwidth = 1,
    dtick = 1,
    range=[0,4],
  )
  ),
  
  legend=dict(
    yanchor="bottom",
    y=-0.4,
    xanchor="center",
    x=0.5,
    bordercolor=axis_color,
    borderwidth=1,
  ),
)

fig.show()

In [22]:
import plotly.graph_objects as go

funcs=['mean', 'std', 'count']
groups=['survey']
columns=['inv_time', 'inv_num', 'inv_amount_average', 'women_led_inv']

df_stats=basic_stats(funcs, groups, columns)

categories=['Investment Experience', 'Number of Investments', 'Average Invested Amount', 'Number of Women-Led Investments']
r1=df_stats[df_stats[groups[0]]=='BA']['mean']
r2=df_stats[df_stats[groups[0]]=='CI']['mean']

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=r1,
      theta=categories,
      fill='toself',
      # fillcolor='rgba(0, 255,0,0.2)',  # Example: 20% opacity
      # line_color='rgba(0, 255,0,0.8)',
      name='Business Angel',
      
))
fig.add_trace(go.Scatterpolar(
      r=r2,
      theta=categories,
      fill='toself',
      # fillcolor='rgba(225, 165,0,0.2)',  # Example: 20% opacity
      # line_color='rgba(225, 165,0,0.8)',
      name='Crowdfunding'
))

axis_color = "rgba(108, 122, 137,1)"
fig.update_layout(
  
  polar=dict(
    bgcolor="rgba(0, 0, 0, 0)",
    angularaxis = dict(
    linewidth = 2,
    showline=True,
    linecolor =axis_color,
    gridcolor =axis_color,
  ),
  
  radialaxis = dict(
    side = "counterclockwise",
    showline = False,
    linewidth = 1,
    gridcolor =axis_color,
    gridwidth = 1,
    dtick = 1,
    range=[0,4],
  )
  ),
  
  legend=dict(
    yanchor="bottom",
    y=-0.4,
    xanchor="center",
    x=0.5,
    bordercolor=axis_color,
    borderwidth=1,
  ),
)

fig.show()

# Investment Experience - inv_time

## Business Angels

Q3.1 How long ago did you become a business angel?
o	Within the last year  (1) 
o	2-3 years ago  (2) 
o	4-6 years ago  (3) 
o	7-9 years ago  (4) 
o	10 or more years ago  (5) 

## Crowfunding Investors

Q3.1 How long ago did you first join a crowdfunding platform?
o	Within the last year  (1) 
o	2-3 years ago  (2) 
o	4-6 years ago  (3) 
o	7-9 years ago  (4) 
o	10 or more years ago  (5) 

## Mapping

1 => < 1y
2 => [2y, 3y]
3 => [4y, 6y]
4 => [7y, 9y]
5 => ≥ 10y



In [68]:
# Statistics - Gender
funcs=[mode, 'median', 'mean', 'std', 'count']
groups=['gender_male']
columns=['inv_time',]

basic_stats(funcs, groups, columns)

,gender_male,mode,median,mean,std,count,var
0,1.0,3.0,3.0,3.064516,1.093480,31,inv_time
1,2.0,5.0,3.0,3.372093,1.337658,86,inv_time


In [69]:
# Statistics - BA/CI
funcs=[mode, 'median', 'mean', 'std', 'count']
groups=['survey']
columns=['inv_time',]

basic_stats(funcs, groups, columns)

,survey,mode,median,mean,std,count,var
0,BA,3.0,4.0,3.563830,1.187375,94,inv_time
1,CI,2.0,2.0,2.173913,1.029217,23,inv_time


In [147]:
map_table=[
        [1,'Within the last year'],
        [2,'2-3 years ago'],
        [3,'4-6 years ago'],
        [4,'7-9 years ago'],
        [5,'10 or more years ago '],
]

title='Investment Experience'
df=df_load
df_display=freq_table(df, 'inv_time', map_table)
df_display.style \
        .set_caption(title)

title='Investment Experience - Women'
df=df_load[df_load['gender_male']==1]
df_display=freq_table(df, 'inv_time', map_table)
df_display.style \
        .set_caption(title)
        
title='Investment Experience - Men'
df=df_load[df_load['gender_male']==2]
df_display=freq_table(df, 'inv_time', map_table)
df_display.style \
        .set_caption(title)

title='Investment Experience - Business Angels'
df=df_load[df_load['survey']=='BA']
df_display=freq_table(df, 'inv_time', map_table)
df_display.style \
        .set_caption(title)

title='Investment Experience - Crowdfunding Investors'
df=df_load[df_load['survey']=='CI']
df_display=freq_table(df, 'inv_time', map_table)
df_display.style \
        .set_caption(title)
        

C:\Users\Dell\AppData\Local\Temp\ipykernel_13492\4111158924.py:33: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

C:\Users\Dell\AppData\Local\Temp\ipykernel_13492\4111158924.py:33: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

C:\Users\Dell\AppData\Local\Temp\ipykernel_13492\4111158924.py:33: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes

,Response,Frequency,Percentage
0,Within the last year,7,0.107692
0,2-3 years ago,8,0.123077
0,4-6 years ago,5,0.076923
0,7-9 years ago,3,0.046154
0,10 or more years ago,0,0.000000
0,NA,42,0.646154
0,Total,65,1.000000


In [303]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

def des_anal(df, main_title, x_title, x_label, y_title, category_range, category_orders):

    ######################## Histograms - One group ########################

    groups={
        'gender_male':{
            'title': main_title + ' - Gender',
            'names': ['Men', 'Women'],
            'values': [1, 2],
        },
        'survey':{
            'title': main_title + ' - Investor Type',
            'names': ['Business Angel', 'Crowdfunding Investor'],
            'values': ['BA', 'CI'],
        },
    }
    for group in groups:
        fig = go.Figure()
        for i in range(len(groups[group]['values'])):    
            x=df[df[group]==groups[group]['values'][i]][x_label].replace(category_range, category_orders)
            fig.add_trace(go.Histogram(histnorm='percent', x=x, name=groups[group]['names'][i]))
    
        # Layout
        fig.update_layout(
            title=groups[group]['title'],
            xaxis={
                'title': x_title,
                'showline': True,
                'linewidth': 1,
                'linecolor': 'black',
                # 'mirror': True,
                'categoryorder': 'array',
                'categoryarray': category_orders,
            },
            yaxis={
                'title': y_title,
                'showline': True,
                'linewidth': 1,
                'linecolor': 'black',
                # 'mirror': True,
            },
        )

        fig.show()
    
    ######################## Histograms - Cross Groups ########################

    groups={
        'gender_male':{
            'title': main_title + ' - Gender',
            'names': ['Men', 'Women'],
            'values': [1, 2],
        },
        'survey':{
            'title': main_title + ' - Investor Type',
            'names': ['Business Angel', 'Crowdfunding Investor'],
            'values': ['BA', 'CI'],
        },
    }
    title = main_title + ' - Gender & Investor Type'
    fig = go.Figure()
    
    for i in range(len(groups['gender_male']['values'])):    
        for j in range(len(groups['survey']['values'])):    
            x=df[(df['gender_male']==groups['gender_male']['values'][i]) & (df['survey']==groups['survey']['values'][j])][x_label].replace(category_range, category_orders)
            fig.add_trace(go.Histogram(histnorm='percent', x=x, name=groups['gender_male']['names'][i]+' & '+groups['survey']['names'][j]))

    # Layout
    fig.update_layout(
        title=title,
        xaxis={
            'title': x_title,
            'showline': True,
            'linewidth': 1,
            'linecolor': 'black',
            # 'mirror': True,
            'categoryorder': 'array',
            'categoryarray': category_orders,
        },
        yaxis={
            'title': y_title,
            'showline': True,
            'linewidth': 1,
            'linecolor': 'black',
            # 'mirror': True,
        },
    )

    fig.show()
    
    ######################## Statistics ########################
    columns=[x_label]

    groups=['gender_male']
    df_stats=basic_stats(df,funcs,groups,columns)
    df_stats['gender_male']=df_stats['gender_male'].replace([1,2], ['Men', 'Women'])
    display(df_stats)
    
    groups=['survey']
    df_stats=basic_stats(df,funcs,groups,columns)
    df_stats['survey']=df_stats['survey'].replace(['BA','CI'], ['Business Angel', 'Crowdfunding Investor'])
    display(df_stats)
    
    groups=['gender_male', 'survey']
    df_stats=basic_stats(df,funcs,groups,columns)
    df_stats['gender_male']=df_stats['gender_male'].replace([1,2], ['Men', 'Women'])
    df_stats['survey']=df_stats['survey'].replace(['BA','CI'], ['Business Angel', 'Crowdfunding Investor'])
    display(df_stats)
    
    ######################## Mann Whitney U Tests ########################
    condition_label='gender_male'
    condition_values=[1,2]
    print('Mann Whitney U Test - Gender')
    display(mw_test(df_load, columns, condition_label, condition_values))
    condition_label='survey'
    condition_values=['BA','CI']
    print('Mann Whitney U Test - Investor Type')
    display(mw_test(df_load, columns, condition_label, condition_values))
    

In [304]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df=df_load.copy()
main_title='Investment Experience'
x_title=''
x_label='inv_time'
y_title='Frequency'
category_range=[1,2,3,4,5]
category_orders=['Within the last year','2-3 years ago', '4-6 years ago', '7-9 years ago', '10 or more years ago']

des_anal(df, main_title, x_title, x_label, y_title, category_range, category_orders)


,gender_male,mode,median,mean,std,total,count,count_na,percent_na,var
0,Men,3.0,3.0,3.064516,1.093480,47,31,16,0.340426,inv_time
1,Women,5.0,3.0,3.372093,1.337658,101,86,15,0.148515,inv_time


,survey,mode,median,mean,std,total,count,count_na,percent_na,var
0,Business Angel,3.0,4.0,3.563830,1.187375,123,94,29,0.235772,inv_time
1,Crowdfunding Investor,2.0,2.0,2.173913,1.029217,65,23,42,0.646154,inv_time


,gender_male,survey,mode,median,mean,std,total,count,count_na,percent_na,var
0,Men,Business Angel,3.0,3.0,3.375000,0.923721,29,24,5,0.172414,inv_time
1,Men,Crowdfunding Investor,1.0,2.0,2.000000,1.000000,18,7,11,0.611111,inv_time
2,Women,Business Angel,5.0,4.0,3.628571,1.264584,72,70,2,0.027778,inv_time
3,Women,Crowdfunding Investor,2.0,2.0,2.250000,1.064581,29,16,13,0.448276,inv_time


Mann Whitney U Test - Gender


C:\Users\Dell\AppData\Local\Temp\ipykernel_13492\802478183.py:18: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



,var,u_stat,pvalue
0,inv_time,1155.5,0.261672


Mann Whitney U Test - Investor Type


C:\Users\Dell\AppData\Local\Temp\ipykernel_13492\802478183.py:18: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



,var,u_stat,pvalue
0,inv_time,1729.5,0.000005


In [24]:
df_load[['inv_criteria', 'inv_sectors']].head()

,inv_criteria,inv_sectors
0,"3,4","1,2"
1,"1,3,4",1
2,"3,4,5","1,3,9"
3,"2,3,4","1,3,5"
4,"2,3,4","3,4,9"


# Investment Criteria

## Business Angels

Q3.11 What are the most important factors you consider before making an investment? (max 3 options)
▢	Alignment with my experience and expertise  (1) 
▢	Business plan  (2) 
▢	Scalability/Market potential  (3) 
▢	Entrepreneurial team  (4) 
▢	Environmental/social impact  (5) 
▢	Exit strategy  (6) 
▢	Fair valuation/return  (7) 
▢	Other (to clarify)  (8) __________________________________________________

## Crowfunding Investors

Q3.8 What are the main factors you consider before making an investment via a crowdfunding platform? (max 3 options)
▢	Business plan  (1) 
▢	Crowd opinion  (2) 
▢	Entrepreneurial team  (3) 
▢	Environmental/social impact  (4) 
▢	Exit strategy  (5) 
▢	Fair valuation/return  (6) 
▢	Other (to clarify)  (7) __________________________________________________

## Consolidated questionaire

1. Business plan
2. Entrepreneurial team
3. Environmental/social impact 
4. Exit strategy
5. Fair valuation/return 
6. Alignment with my experience and expertise
7. Scalability/Market potential
8. Crowd opinion
9. Other


,gender_male,mode,median,mean,std,count,var
0,1.0,3.0,3.0,3.064516,1.093480,31,inv_time
1,2.0,5.0,3.0,3.372093,1.337658,86,inv_time


In [ ]:
# Statistics - Gender
funcs=[mode, 'median', 'mean', 'std', 'count']
groups=['gender_male']
columns=['inv_time',]

basic_stats(funcs, groups, columns)

In [ ]:
# inv_sectors

# BA
# Q3.12 In which business sector(s) have you invested as a business angel?
# ▢	Information technology (software, internet, IT services, etc.)  (1) 
# ▢	Finance  (2) 
# ▢	Healthcare / Biotech  (3) 
# ▢	Manufacturing  (4) 
# ▢	Retail  (5) 
# ▢	Consulting  (6) 
# ▢	Energy  (7) 
# ▢	Transportation  (8) 
# ▢	Other (please clarify)  (9) __________________________________________________

# CI
# Q3.9 In which business sector(s) have you invested as a crowd investor?
# ▢	Information technology (software, internet, IT services, etc.)  (1) 
# ▢	Finance  (2) 
# ▢	Healthcare / Biotech  (3) 
# ▢	Manufacturing  (4) 
# ▢	Retail  (5) 
# ▢	Consulting  (6) 
# ▢	Energy  (7) 
# ▢	Transportation  (8) 
# ▢	Other (please clarify)  (9) __________________________________________________
